# CAT synthetic data generation

Due to time constraints, we generated synthetic data for our time series for the counter attack turret (CAT) system. Below is a function to generate 10000 rows of data we needed to simulate a cat jumping on a counter.

In [3]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta, time

def generate_cat_counter_data(num_rows=1000, start_date=datetime(2023, 1, 1)):
    """Generates synthetic time series data with adjusted counter access probability."""

    timestamps = [start_date + timedelta(minutes=i) for i in range(num_rows)]

    is_counter_access_attempted = []
    last_feeding_time = []
    next_feeding_time = []

    feeding_times = [time(8, 0), time(15, 0), time(21, 0)]  # 8 AM, 3 PM, 9 PM

    for timestamp in timestamps:
        # Determine the last and next feeding times
        last_feeding = None
        next_feeding = None
        for feeding_time in feeding_times:
            feeding_datetime = datetime.combine(timestamp.date(), feeding_time)
            if feeding_datetime <= timestamp:
                last_feeding = feeding_datetime
            else:
                next_feeding = feeding_datetime
                break

        if last_feeding is None:
            last_feeding = datetime.combine(timestamp.date() - timedelta(days=1), feeding_times[-1])
        if next_feeding is None:
            next_feeding = datetime.combine(timestamp.date() + timedelta(days=1), feeding_times[0])

        last_feeding_time.append(last_feeding)
        next_feeding_time.append(next_feeding)

        # Adjusted Counter access simulation
        time_since_last_feeding = timestamp - last_feeding
        time_until_next_feeding = next_feeding - timestamp

        # Probability increases linearly as time since the last feeding increases,
        # and reaches its maximum just before the next feeding.
        access_probability = time_since_last_feeding.total_seconds() / (next_feeding - last_feeding).total_seconds()

        # Add some randomness
        access_probability += random.uniform(-0.1, 0.1)
        access_probability = max(0, min(1, access_probability))

        is_counter_access_attempted.append(random.random() < access_probability)

    data = {
        'timestamp': timestamps,
        'is_counter_access_attempted': is_counter_access_attempted,
        'last_feeding_time': last_feeding_time,
        'next_feeding_time': next_feeding_time
    }
    df = pd.DataFrame(data)
    return df

# Generate and display the data
cat_data = generate_cat_counter_data(2000)
print(cat_data.head(20))
cat_data.to_csv("cat_counter_data.csv", index=False)

             timestamp  is_counter_access_attempted   last_feeding_time  \
0  2023-01-01 00:00:00                        False 2022-12-31 21:00:00   
1  2023-01-01 00:01:00                        False 2022-12-31 21:00:00   
2  2023-01-01 00:02:00                        False 2022-12-31 21:00:00   
3  2023-01-01 00:03:00                        False 2022-12-31 21:00:00   
4  2023-01-01 00:04:00                        False 2022-12-31 21:00:00   
5  2023-01-01 00:05:00                        False 2022-12-31 21:00:00   
6  2023-01-01 00:06:00                         True 2022-12-31 21:00:00   
7  2023-01-01 00:07:00                         True 2022-12-31 21:00:00   
8  2023-01-01 00:08:00                         True 2022-12-31 21:00:00   
9  2023-01-01 00:09:00                         True 2022-12-31 21:00:00   
10 2023-01-01 00:10:00                         True 2022-12-31 21:00:00   
11 2023-01-01 00:11:00                        False 2022-12-31 21:00:00   
12 2023-01-01 00:12:00   